In [2]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [133]:
import sqlite3
import os

import pandas as pd
import numpy as np
from src.utils import get_tables, df_na_vals
from src.vast_api import df_to_tmp_table, get_machines, get_offers, _get_raw, preprocess, np_group_by

In [140]:
raw = _get_raw('https://500.farm/vastai-exporter/offers')

In [142]:
raw.machine_id.size, raw.machine_id.unique().size

(5365, 1218)

In [97]:
offers = get_offers()

In [98]:
offers.machine_id.size, offers.machine_id.unique().size

(3299, 1220)

In [150]:
machine_ids, max_chunk = np_group_by(raw, 'num_gpus', np.maximum)
max_chunk = pd.Series(max_chunk, index=machine_ids)
max_chunk

162      4
274      8
275      8
283      6
301      8
        ..
11834    1
11835    1
11836    1
11837    1
11840    5
Length: 1218, dtype: int64

In [63]:
%timeit np_group_by(raw, 'num_gpus', np.maximum)

331 µs ± 7.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [145]:
df = get_machines()

In [146]:
df.machine_id.size

1213

In [147]:
max_chunk = max_chunk[max_chunk.index.isin(df.machine_id)]

In [149]:
max_chunk.sort_index()

162      4
274      8
275      8
283      6
301      8
        ..
11834    1
11835    1
11836    1
11837    1
11840    5
Length: 1213, dtype: int64

In [148]:
(df.set_index('machine_id').loc[max_chunk.index, 'num_gpus'] == max_chunk).all()

True

In [152]:
max_chunk[~max_chunk.index.isin(df.machine_id)]

3690      7
5919      4
6312      4
8384     11
11693    12
dtype: int64

In [159]:
# raw.loc[raw.machine_id == 3690, ['num_gpus', 'verified', 'rentable']]
raw.loc[raw.machine_id == 3690]

,bundle_id,bw_nvlink,compute_cap,cpu_cores,cpu_cores_effective,cpu_name,cpu_ram,credit_balance,credit_discount,credit_discount_max,...,reliability2,rentable,score,start_date,static_ip,storage_cost,total_flops,verification,verified,timestamp
250,314689779,0.0,860,32.0,32.000000,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,64.169220,NaN,True,0.25,310.15680,verified,True,1690652116
736,314689780,0.0,860,32.0,18.285728,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,84.933244,NaN,True,0.25,176.96256,verified,True,1690652116
1001,314689777,0.0,860,32.0,13.714272,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,80.170319,NaN,True,0.25,133.19424,verified,True,1690652116
1002,314689777,0.0,860,32.0,13.714272,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,81.377691,NaN,True,0.25,133.19424,verified,True,1690652116
1012,314689777,0.0,860,32.0,13.714272,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,78.729211,NaN,True,0.25,132.24960,verified,True,1690652116
2611,314689778,0.0,860,32.0,4.571424,AMD Ryzen Threadripper PRO 3955WX 16-Cores,257566,None,None,0.1,...,0.997692,False,76.957123,NaN,True,0.25,44.71296,verified,True,1690652116


In [21]:
df_na_vals(df)

bw_nvlink                 ->          Missing values: 0 (0.00%)
compute_cap               ->          Missing values: 0 (0.00%)
cpu_cores                 ->          Missing values: 5 (0.41%)
cpu_name                  ->          Missing values: 10 (0.83%)
cpu_ram                   ->          Missing values: 0 (0.00%)
credit_balance            ->          Missing values: 1212 (100.00%)
credit_discount           ->          Missing values: 1212 (100.00%)
credit_discount_max       ->          Missing values: 257 (21.20%)
cuda_max_good             ->          Missing values: 0 (0.00%)
direct_port_count         ->          Missing values: 0 (0.00%)
discount_rate             ->          Missing values: 257 (21.20%)
discounted_dph_total      ->          Missing values: 0 (0.00%)
discounted_hourly         ->          Missing values: 0 (0.00%)
disk_bw                   ->          Missing values: 1 (0.08%)
disk_name                 ->          Missing values: 0 (0.00%)
disk_space             

['credit_balance', 'credit_discount']

In [25]:
df.disk_bw

0               NaN
1       1684.000000
2       2626.785612
3       1749.841967
4       1220.300939
           ...     
1207    1933.792757
1208     492.589678
1209    1476.081213
1210    1400.685966
1211    2089.399414
Name: disk_bw, Length: 1212, dtype: float64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   bw_nvlink            1210 non-null   int32 
 1   compute_cap          1210 non-null   int32 
 2   cpu_cores            1210 non-null   int32 
 3   cpu_name             1210 non-null   object
 4   cpu_ram              1210 non-null   int32 
 5   credit_discount_max  1210 non-null   int32 
 6   cuda_max_good        1210 non-null   object
 7   direct_port_count    1210 non-null   int32 
 8   disk_bw              1210 non-null   int32 
 9   disk_name            1210 non-null   object
 10  disk_space           1210 non-null   int32 
 11  dlperf               1210 non-null   int32 
 12  dph_base             1210 non-null   int32 
 13  driver_version       1210 non-null   object
 14  end_date             1210 non-null   int32 
 15  gpu_display_active   1210 non-null   int32 
 16  gpu_la

In [2]:
os.getcwd()

'C:\\Users\\ergot\\DataspellProjects\\vast'

In [3]:
conn = sqlite3.connect(f'test/test.db')
# conn = sqlite3.connect(f'sqlite.db')


In [10]:
df = pd.DataFrame([{'id': 3, 'mid': 0}])
df_to_tmp_table(df, 'tmp', conn)

In [11]:
get_tables(conn)

['map_tbl', 'tmp']

In [13]:
conn.execute('SELECT * FROM tmp').fetchall()[0]

(3, 0)

In [4]:
conn.execute(f'''
SELECT
    name
FROM
    sqlite_master
WHERE
    type ='table' AND
    name NOT LIKE 'sqlite_%';
''').fetchall()

[('map_tbl',)]

In [65]:
conn.close()

In [18]:
pd.DataFrame([{'id': 3, 'mid': 0}])

,id,mid
0,3,0


In [ ]:
get_tables()